In [ ]:
# Install required libraries
!pip install git+https://github.com/PrithivirajDamodaran/Gramformer.git -q
!pip install gradio -q
!pip install nltk -q

In [ ]:
# Import libraries
import gradio as gr
import nltk
from gramformer import Gramformer
from nltk.tokenize import sent_tokenize
import warnings
import os

# Suppress FutureWarning messages from transformers
warnings.filterwarnings('ignore', category=FutureWarning)

# Download NLTK data for sentence tokenization
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

In [ ]:
# Initialize Gramformer (no GPU for simplicity)
gf = Gramformer(models=1, use_gpu=False)

# Function to correct a single sentence
def correct_sentence_grammar(sentence):
    try:
        corrections = gf.correct(sentence, max_candidates=1)
        for corrected in corrections:
            if corrected.strip() != sentence.strip():
                return corrected
        return sentence
    except Exception as e:
        print(f"Gramformer error: {e}")
        return sentence

# Function to process text (split into sentences and correct)
def process_text(text):
    sentences = sent_tokenize(text)
    corrected_sentences = [correct_sentence_grammar(sentence) for sentence in sentences]
    return sentences, corrected_sentences

# Function to generate HTML table for results
def generate_results_html(original_sentences, corrected_sentences):
    html = "<h3>Results</h3><table><tr><th>Before</th><th>After</th></tr>"
    for orig, corr in zip(original_sentences, corrected_sentences):
        html += f"<tr><td style='color:red'>{orig}</td><td style='color:green'>{corr}</td></tr>"
    html += "</table>"
    return html

# Main function for the Gradio app
def correct_text(text, file):
    try:
        # If a file is uploaded, read its content
        if file is not None:
            with open(file.name, 'r', encoding='utf-8') as f:
                text = f.read()
        # Check if text is empty
        if not text.strip():
            return "Please enter text or upload a .txt file."
        # Process the text
        original_sentences, corrected_sentences = process_text(text)
        # Return results as HTML
        return generate_results_html(original_sentences, corrected_sentences)
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
interface = gr.Interface(
    fn=correct_text,
    inputs=[
        gr.Textbox(lines=4, placeholder="Enter text here..."),
        gr.File(label="Upload .txt file")
    ],
    outputs=gr.HTML(),
    title="Natural Language Correction System",
    description="Correct your text using advanced NLP models."
)

# Launch the app quietly with a public URL
interface.launch(share=True, debug=False, show_error=False, quiet=True)